<a href="https://colab.research.google.com/github/dagnybrand/AI-news-bias/blob/main/train_on_all_inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os.path
import json
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
import sklearn.model_selection

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
if not os.path.exists('bias_data_short.json'):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rF2ZPWDw2_N4E6Uo5SGG8W6YazuTDXnz' -O bias_data_short.json

In [159]:

with open('bias_data_short.json') as f:
  full_data_dict = json.loads(f.read())

headers = ['topic', 'source', 'bias', 'title', 'content']
df = pd.DataFrame()

topics = []
source = []
bias = []
title = []
content = []

lists = [topics, source, bias, title, content]

for item in full_data_dict:
  for header, data in zip(headers, lists):
    if item[header] is not None:
      data.append(item[header])
    else:
      data.append('null')

for header, data in zip(headers, lists):
  df[header] = data

y_list = list(df['bias'])

x_df = df.drop(labels='bias', axis=1)

In [ ]:
# tried count vectorizer method from sklearn but does not work as well as universal encoder so sticking with that
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_df['title'])

print(X_train_counts)
X_train_counts.shape

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
topic_embeddings = model(x_df['topic'])
content_embeddings = model(x_df['content'])
source_embeddings = model(x_df['source'])
title_embeddings = model(x_df['title'])

In [ ]:
em_df = pd.DataFrame()
em_df['topic'] = np.array(topic_embeddings).tolist()
em_df['content'] = np.array(content_embeddings).tolist()
em_df['title'] = np.array(title_embeddings).tolist()
em_df['source'] = np.array(source_embeddings).tolist()

In [ ]:
print(em_df)

                                                  topic  \
0     [-0.0628657415509224, -0.04393262043595314, -0...   
1     [-0.037389691919088364, -0.0498385950922966, 0...   
2     [0.017720667645335197, -0.015966027975082397, ...   
3     [-0.0628657415509224, -0.04393262043595314, -0...   
4     [-0.0628657415509224, -0.04393262043595314, -0...   
...                                                 ...   
996   [-0.061572831124067307, -0.006184494588524103,...   
997   [-0.025700176134705544, -0.038757916539907455,...   
998   [0.0036468892358243465, 0.033641085028648376, ...   
999   [-0.028081052005290985, -0.005823448300361633,...   
1000  [0.06156580522656441, -0.033410198986530304, 0...   

                                                content  \
0     [0.005978220608085394, -0.05688862502574921, -...   
1     [-0.022267526015639305, -0.05396590754389763, ...   
2     [-0.023165499791502953, -0.05102813243865967, ...   
3     [-0.032482728362083435, -0.05443748086690903, ...

In [146]:
em_df_exp = pd.DataFrame()

count = 0

import warnings
warnings.filterwarnings("ignore", message="DataFrame is highly fragmented.")

# expand data frame so each coefficient of input vector for each category has a column
for col in em_df.columns:
  data = {}
  for item in em_df[col]:
    count = 0
    for element in item:
      if count in data:
        data[count].append(element)
      else:
        data[count] = [element]
      count += 1

  for key in data:
    em_df_exp[f'{col}{key}'] = data[key].copy()



In [110]:
print(em_df_exp)

        topic0    topic1    topic2    topic3    topic4    topic5    topic6  \
0    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
1    -0.037390 -0.049839  0.055891  0.020852 -0.060803 -0.034120  0.023585   
2     0.017721 -0.015966  0.012558  0.053298  0.013441 -0.015035 -0.030523   
3    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
4    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
...        ...       ...       ...       ...       ...       ...       ...   
996  -0.061573 -0.006184 -0.028643  0.067156 -0.079502  0.026094 -0.010935   
997  -0.025700 -0.038758  0.075657  0.037729 -0.036311  0.027566  0.062982   
998   0.003647  0.033641  0.087970 -0.000100 -0.017960  0.041255 -0.026474   
999  -0.028081 -0.005823  0.075380  0.014799 -0.039165  0.029606 -0.022766   
1000  0.061566 -0.033410  0.062320  0.083477 -0.003510  0.020073  0.051909   

        topic7    topic8    topic9  ...  source502  source503  

In [149]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(em_df_exp, y_list, test_size=0.3, random_state=42)

In [152]:
sgd = SGDClassifier()

sgd.fit(X_train, y_train)

score = sgd.score(X_test, y_test)
print(score)

preds = sgd.predict(X_test)

for i in range(len(preds)):
  if preds[i] != y_test[i]:
    print(f"mispredicted {y_test[i]} as {preds[i]} ")



0.9235880398671097
mispredicted 0 as 1 
mispredicted 1 as 2 
mispredicted 2 as 1 
mispredicted 0 as 2 
mispredicted 0 as 1 
mispredicted 0 as 2 
mispredicted 2 as 0 
mispredicted 1 as 2 
mispredicted 0 as 2 
mispredicted 2 as 1 
mispredicted 0 as 2 
mispredicted 0 as 2 
mispredicted 0 as 1 
mispredicted 1 as 2 
mispredicted 1 as 2 
mispredicted 0 as 2 
mispredicted 0 as 2 
mispredicted 0 as 2 
mispredicted 0 as 2 
mispredicted 1 as 2 
mispredicted 0 as 2 
mispredicted 1 as 2 
mispredicted 0 as 2 


In [155]:
# function to get test data into classifier input

def prep_test(test):
  import warnings
  warnings.filterwarnings("ignore", message="DataFrame is highly fragmented")

  # embedded text input
  em_topic = model(test['topic'])
  em_title = model(test['title'])
  em_content = model(test['content'])
  em_source = model(test['source'])

  # get embedded df
  em_test = pd.DataFrame()
  em_test['topic'] = np.array(em_topic).tolist()
  em_test['content'] = np.array(em_content).tolist()
  em_test['title'] = np.array(em_title).tolist()
  em_test['source'] = np.array(em_source).tolist()

  # get expanded data frame
  em_test_exp = pd.DataFrame()

  count = 0

  # expand data frame so each coefficient of input vector for each category has a column
  for col in em_test.columns:
    data = {}
    for item in em_test[col]:
      count = 0
      for element in item:
        if count in data:
          data[count].append(element)
        else:
          data[count] = [element]
        count += 1

    for key in data:
      em_test_exp[f'{col}{key}'] = data[key].copy()

  return em_test_exp

In [154]:

def predict(input):
  preds = sgd.predict(input)
  text_preds = []
  for pred in preds:
    if (pred == 0):
      text_preds.append('left')
    if (pred == 1):
      text_preds.append('middle')
    if (pred == 2):
      text_preds.append('right')
  return text_preds

In [130]:
# test on found article https://www.cnn.com/2023/11/12/politics/kevin-mccarthy-matt-gaetz-mike-johnson-house-speaker/index.html

test = pd.DataFrame()

test['title'] = ['An aggrieved McCarthy weighs leaving the House with his GOP foes on his mind']
test['source'] = ['CNN']
test['content'] = ["Former House Speaker Kevin McCarthy is weighing whether to stay in office following his unprecedented ouster as speaker – even as he believes he didn’t deserve to lose his job and is still resentful of the Republicans who effectively ended his nine-month tenure atop the House. In an exclusive interview with CNN, McCarthy made clear he misses being “at the table” even as he promised to help the new speaker; went further in his embrace of former President Donald Trump heading into 2024; and questioned whether the “crazy eight” Republicans who voted to strip him of the gavel are even conservatives at all. The California Republican reserved his harshest attacks for Rep. Matt Gaetz of Florida, who led the charge for McCarthy’s ouster. McCarthy continually accused Gaetz of only going after him because of a House Ethics Committee probe he’s facing, a charge the GOP hardliner has repeatedly denied. 'I don’t believe them to be conservatives,' McCarthy said of the eight Republicans. 'It’s driven by Gaetz, and it was all based upon an ethics complaint that happened in the last Congress. He would throw his country away to try to protect himself for what would come out as the truth.' McCarthy suggested that Gaetz had called for the vote for McCarthy’s removal – and later backed Mike Johnson’s ascension as speaker – simply to quash the ethics probe. 'If the Ethics Committee never does anything to Gaetz, then Gaetz was successful in stopping probably what rightfully should come to him,' McCarthy said. While McCarthy called for Gaetz to face “consequences,” accusing him of having violated internal House GOP Conference rules by calling the ouster vote without the requisite support from Republicans, Johnson doesn’t seem to be going in that direction. Asked if he were concerned about Johnson’s approach to Gaetz, McCarthy said: 'That’s a question for Speaker Johnson. I don’t know if they made any deals for the vote or anything like that. I don’t believe so. But I will let Ethics do their work.' Johnson’s office declined to comment, while Gaetz brushed off the attacks. 'Thoughts and prayers to the former speaker as he works through his grief,' Gaetz said. McCarthy, adjusting from being the most powerful Republican in Washington to a rank-and-file member not driving his party’s agenda, is planning to fill out the rest of his term, his aides say. But he made clear in the interview that he may call forgo a reelection bid.'I got the holidays. I will talk to my family about the ideas of what is going forward, and then I will make a decision,' McCarthy told CNN as he sat in his small office, which he noted was where Harry Truman was playing cards upon learning he’d become president in 1945 following Franklin Roosevelt’s death. Yet McCarthy sounded uncertain about staying in the House. 'Well, there’s so many ways you can do that to make sure you’re getting the job done. And I’m going to look at all options,' he added. When asked if he was leaning toward staying in Congress behind or choosing a new path, McCarthy said, 'I don’t know.' Meanwhile, McCarthy’s removal as speaker may be felt in the GOP’s pocketbook. In his time as GOP leader and then as speaker, he was a fundraising juggernaut for the party, raking in $637 million for his two affiliated outside groups, the Congressional Leadership Fund and the American Action Network. And federal records show that McCarthy raised $78 million for four of his fundraising entities just this year. While Johnson has already taken steps to bolster his fundraising appeal – and as the GOP’s party committee touts strong fundraising since his election as speaker –the Louisiana Republican has reached out to McCarthy to help introduce him to big donors. 'The speaker called me last night, asking if I would go to an event,' McCarthy said Thursday. 'I’m going to do everything I can to introduce him to people … because it’s for the conference and the country itself.' But McCarthy has maintained a low profile since Johnson’s election, saying he hasn’t been attending GOP conference meetings 'because I don’t want people to turn to me' as the new speaker begins his tenure. McCarthy told CNN that he has not spoken to House Minority Leader Hakeem Jeffries since his ouster, although the two maintained a cordial relationship during his time as speaker — before the New York Democrat voted for his removal. And he dismissed concerns from former Speaker Nancy Pelosi, who lost valuable office space in the Capitol after he greenlit ejecting her from the space. 'Well, you now have a new former speaker,' he said. 'That’s the problem.' McCarthy also accused Pelosi of betraying her word when it came to the motion to oust him. 'Before I was elected, I was still having problems with getting the votes, which she did as well. And I told her the issue was bringing back this motion to vacate. And the first thing she said: 'Just give it to them. Just give it to them. We’d never allow that. It’s not good for the House,' McCarthy said of Pelosi. 'Look, at the end of the day, the Democrats decided to make a political decision.' Aaron Bennett, a Pelosi spokesman, called McCarthy’s comments 'nonsense.' 'Speaker Pelosi has made clear – to McCarthy and to everyone – that she values the dignity of the House,' Bennett said. 'Recognizing that Hakeem Jeffries is our candidate for speaker, Speaker Pelsoi said to McCarthy that we will take our lead from him.' But it was McCarthy’s acquiescence to the hard right that ultimately did him in. As he was scrambling to get the votes to be elected speaker in January – ultimately gaining the position after 15 ballots – he agreed to lower the threshold to allow a single member to call for a vote ousting a sitting speaker, a departure from the threshold under Pelosi, who had made it much harder to call for such a vote. Asked if he had thought he would make it two years as speaker, McCarthy said: 'I never knew.' McCarthy credited Trump for his winning the gavel. But in recent months, he came under hot water from Trump’s team when he seemed to question whether he would be the best candidate for the GOP. In the interview, McCarthy predicted that the former president will be the party’s nominee next year and said that he will support him. 'I haven’t endorsed, but I support President Trump,' he said. McCarthy has repeatedly defended the former president despite countless controversies and told CNN he is unconcerned about Trump’s criminal charges heading into 2024. McCarthy said 'in most candidates, you would worry about things like that,' but not when it comes to Trump. Still, But he acknowledged it 'could hurt a little' when it comes to Trump’s standing among suburban or swing voters: 'But Biden’s in a worse situation, right?' McCarthy also made clear that he has few regrets – and that he would not change his decision, on the brink of a government shutdown, to advance a stopgap funding bill that needed Democratic support to become law. Gaetz cited the move as the last straw. 'I would do the exact same thing again to keep our government open, to fund our troops,'' McCarthy said before unloading on the eight Republicans. 'This country is too great to have such small ideas like these eight have,' McCarthy said. And he continued to question Gaetz’s motivations. 'He doesn’t have a conservative bent in his philosophy, and just the nature of what he focuses on,'' McCarthy said of Gaetz, recalling an incident in which the 41-year-old Florida Republican discussed what drives him. 'The TV congressman – that’s what he wants to be known for.'"]
test['topic'] = ['politics']

em_test = prep_test(test)

In [128]:
# predicts left!
print(predict(em_test))

['left']


In [156]:
# predicted right for https://www.foxnews.com/media/david-axelrod-warns-bidens-age-issue-consistent-concern-among-voters-one-thing-you-cant-reverse

test = pd.DataFrame()

test['title'] = ["David Axelrod warns Biden's 'age issue' is consistent concern among voters: 'One thing you can't reverse'"]
test['source'] = ['Fox News']
test['content'] = ["Former Obama adviser David Axelrod warned on Sunday that President Biden's 'age issue' was consistent in polling and added that it was the 'one thing' they can't reverse, 'no matter how effective' the president is behind the scenes. Axelrod joined CNN's Dana Bash and former Republican Maryland Gov. Larry Hogan on Sunday to discuss a potential third-party run by Sen. Joe Manchin, D-W.Va., who recently announced that he wouldn't be seeking re-election in the Senate. Hogan said he would go a step further than Axelrod, who suggested last week the president should consider whether it would be 'wise' for him to drop out of the race, and said Biden might not be the strongest nominee for the Democratic Party. 'I have no concerns about polls a year out. I mean, you have to look at them and analyze them and adjust. But I was in a situation as a strategist for Barack Obama in 2011 where we were facing some difficult polls. The one number in the polling that was concerning, and in the CNN poll that followed after The New York Times poll, had to do with age, and that is one thing you can’t reverse no matter how effective Joe Biden is behind the scenes. In front of the camera, what he’s projecting is causing people concerns, and that is worrisome,' Axelrod told Bash and Hogan. Axelrod also said Hogan needed to do a better job of preventing Donald Trump from getting the Republican nomination. Bash asked Axelrod about how a third-party candidate might hurt Biden and help Trump, but also noted Rep. Dean Phillips, D-Minn., who is challenging Biden in the Democratic primary. 'Look, I said what I said. I think that there is one issue that is hanging over him. The country is in a sour mood, no doubt about it. It is hard to be an incumbent, and it's been in a sour mood since before he was elected. There are concerns about inflation, that's a problem. Those could be overcome. And I think with Donald Trump on the other end, he could still win this election. But the age issue is difficult. And, so, Phillips is trying to exploit that,' Axelrod said. 'I believe Joe Biden will be the nominee of the Democratic Party. And right now, we're getting to the third party point. My concern is that Donald Trump has a high floor and a low ceiling, and you throw a bunch of third party candidates in there, and you are making it much more likely that he wins the election,' he added. Media pundits have repeatedly asked the former adviser to President Obama if Biden should drop out of the race. 'I want him to consider what is best in terms of the goal that I know he is committed to, which is defeating Donald Trump. And if he believes, based on not just what's in his heart, but what’s in the data and what he’s being told, that he has the best chance to do it, then he should run. But you know, the thing that irritates me a little bit, Wolf, is this notion that people who are concerned are 'bedwetters,'' Axelrod told CNN's Wolf Blitzer on Monday. CNN's survey, published on Monday, found that 74% of voters do not believe the president 'has the stamina and sharpness to be president.' Only 47% of voters said they do not believe Trump 'has the stamina and sharpness to be president,' the poll found."]
test['topic'] = ['politics']

em_test = prep_test(test)
print(predict(em_test))

['right']


In [158]:
# predicts right on https://www.foxnews.com/media/democratic-presidential-candidate-declares-bidenomics-isnt-working-it-represents-inflation-high-prices

test = pd.DataFrame()

test['title'] = ["Democratic presidential candidate declares 'Bidenomics' isn't working: 'It represents inflation, high prices'"]
test['source'] = ['Fox News']
test['content'] = ["Rep. Dean Phillips, D-Minn., who is challenging President Biden in the Democratic presidential primary, argued that 'Bidenomics' wasn't working and said it represented inflation and 'high prices.' 'Bidenomics is not working. It is actually the opposite. It represents inflation, high prices,' he said. CNN's Dana Bash aired a small portion of the interview on Sunday during 'State of the Union.' 'I respect the president, I want to make it very clear, he's a good man, he saved this country. I think in 2020, he was probably the only Democrat who could have beaten Donald Trump. I think in 2024, he may be among the only ones that will lose to him. And that’s why I’m doing this,' Phillips said during the interview with CNN's Kasie Hunt in New Hampshire. Phillips launched a Democratic primary challenge in late October, after repeatedly criticizing the president for refusing to pass the torch on to the next generation of Democratic leadership. Phillips also told CNN that the president's approval numbers were very low and rivaled Jimmy Carter, who went on to lose his re-election campaign. '[Carter] had a Democratic primary challenger because he was going to get slaughtered in the election. It’s the same circumstance again. Ted Kennedy did not cause Jimmy Carter’s problems. I did not cause President Biden’s problems,' Phillips said, according to CNN. The Minnesota congressman has faced criticism from prominent Democrats, such as Rep. Jim Clyburn, D-S.C., who said Phillips was disrespecting Black voters by announcing his candidacy and campaigning in New Hampshire. 'Black voters are an essential piece of the Democratic coalition, and any candidate for president of the United States ought to know and respect their pivotal role in how Democrats win national elections,' Clyburn argued. Phillips said he would drop out and support the Democratic nominee if he's not ahead on Super Tuesday. 'I’m running in a primary. I’m not doing what Cornel West is doing, not doing what Jill Stein is doing, not doing what Joe Manchin might do,' Phillips told CNN. 'If the president beats me in New Hampshire and Michigan, all around the country, that’s good. He looks strong. If I force the president to have to get out and campaign basically for the first time — because he didn’t have to campaign in 2020 — and he appears well, and he debates, and he’s strong, makes a good case, that’s good for him.'"]
test['topic'] = ['politics']

em_test = prep_test(test)
print(predict(em_test))

['right']
